# Abstractive Text Summarization Sequence-to-sequence RNNs and Beyond
Nallapati et al

## 0. あぶすと
本研究ではアテンションエンコーダー・デコーダーRNNを用い, 概要的な文書要約を行うモデルを設計し, このモデルにより２つの異なるコーパスにおいて群を抜いた性能を達成した  
要約において, 従来のモデルでは適切にモデル化できない致命的な問題――キーワードのモデル化, 文から単語への階層構造の取得, また訓練時にまれであったか, まったく表れなかった単語の出現などに対処するためのいくつかの新しいモデルの提案を行う.  
これらの提案モデルの多くは性能のさらなる改善に貢献したことを示した  
さらに, 複数の文書の要約を含む新しいデータセットの提案と, さらなる研究のための性能評価方法を確立する  


## 1. いんとろ
文書要約は内容の特徴を捉えた少ない文から成る見出しや短い要約を生成するタスク  
ここで, "abstractive"はソースから抽出されたいくつかの節または文の選択による要約ではなく, 文書の主な内容を圧縮するような言い換えである要約であることを表す  
このタスクは文書ソースに含まれる入力文と出力文の対応を取るのがクッソコストかかる  
そんな中最近深層学習ベースの入力系列と出力系列の対応を取るモデルが出てきた(seq2seq)  
んで機械翻訳やら発話認識やらビデオキャプションでめっちゃいい感じになってる  

マチン翻訳とようやくは似てるようでだいぶ違う  
MTとは違ってターゲット(要約)文はめっちゃ短い上にソース文書の長さにはあまり関わらない  
加えて, まちん翻訳では損失を最小にするよう翻訳を予測する一方で, 要約における重要な課題は元文書の重要な概念が保持されるよう, 損失のある方法で最適に圧縮することにある  
翻訳では, ソースとターゲット間における単語レベルでの並びの一対一対応が重視されるが, 要約ではそれは自明  

本研究における主題は以下のとおりである  
1. 既存の機械翻訳向けアテンションエンコーダーデコーダーRNNモデルを要約に適用し, それが異なる2つの英文コーパスにおいて抜きん出たシステムであることを示す  
1. 機械翻訳ベースのモデルでは十分に対処しきれていない具体的な問題について. これらの改善を含む新しいモデルの提案と, さらなる性能の改善を示す  
1. 抽象要約のための新しいデータセットの提案  



## 2. モデル
### 2.1 アテンション付きエンコーダーデコーダーRNNとLarge Vocaburally Trick
基本モデルはBahdanau(2014)の用いたニューラル翻訳モデルに関連している  
エンコーダーモデルは双方向GRU-RNNから成り, デコーダーモデルはエンコーダーモデルと同じだけのサイズの隠れ状態から形成される一方向のGRU-RNNとアテンション機構, ターゲットの語彙から単語を生成するソフトマックスによって構成される.  

この基本モデルに加え, 要約の問題に対処するためLarge Vocaburally Trick(LVT)を導入した.  
本研究におけるアプローチでは, 各ミニバッチにおけるデコーダーの語彙サイズは, そのミニバッチ内に含まれるソース文書の単語に限定される.  
さらに, ターゲット辞書内でもっとも頻度の高い単語が, 語彙が規定の数に達するまで追加される.  
この技術を用いることで, 主要な計算においてボトルネックとなるソフトマックスのサイズを縮小できる  
加えて, この手法は与えられた例において必要となる単語でのみモデル化することにより, 収束を高速化させる  
この手法は要約において特に適しており, なぜなら要約における単語の大部分はどのような場合であってもソース文書から引用されるからである  

### 2.2 Feature-richエンコーダを用いたキーワードの特定  
要約において重要な課題の一つは, 文書内の主要な概念とエンティティを特定することである  
この目標を達成するためには, 入力文書における単語エンベッティングベースの表現をさらに広げ, 単語の品詞タグ・名前付きのエンティティタグといった言語的特徴、またTF-IDFといった統計的特徴を捉えることが必要である  
したがって, 単語埋め込みと同様に, 各タグタイプの語彙のために追加のルックアップテーブルベースの埋め込み行列を作成した  
TF-IDFといった連続的な特徴の場合, それらをカテゴリ値として定められた数のbinに離散化することで変換し, one-hot表現を用いいずれかのbin番号に射影する  
これにより, 他のタグタイプと同様に埋め込み行列にマップすることが可能となる.
最後に, ソース文書の各単語について, 図1に示すように関連するすべてのタグから埋め込みを探し単一の長いベクトルに連結する.
ターゲット側では, 引き続き表現として単語ベースの埋め込みのみ使用する  

### 2.3 スイッチングジェネレータ-ポインターを用いた低頻度/非出現単語のモデリング
要約ではよく, 要約の中心となるキーワードや名前付きエンティティが, 訓練データ内では実際には出現しないか低頻度な場合がある  
なぜなら, デコーダの語彙は訓練時に固定されるため, これらの非出現単語を出すことができない  
代わりにこれらの語彙外単語(OOV)単語を処理するための最も一般的な方法はプレースホルダとして"UNK"トークンを発生させるといったものがある  
しかし, これらは読みやすい要約にはならない.
要約において, このようなOOV単語を扱う直感的な方法としては, 代わりにソース文書内のその場所を指し示すことが必要(OOVトークン→ソース文書におけるUNKに置き換えられた単語を指し示すポインター)  

我々は, 図2に図示されている新しいスイッチングデコーダ/ポインター構造を使用し, この概念をモデル化する  
このモデルでは, すべての時間ステップでgeneratorかポインターを使用するかを決定する｢スイッチ｣がデコーダに実装されている  
スイッチがオォンにされている場合, デコーダは通常通りターゲット語彙から単語を生成する.  
しかし, スイッチがオフの場合, デコーダは代わりにソース内の単語位置を示す一つのポインターを生成する.  
要約にはポインターの位置が指し示す単語がコピーされる  
スイッチは以下に示すように各時間ステップで利用可能な文脈全体に基づいて線形レイヤー上でシグモイド活性化関数としてモデル化されている  

$$ P(s_i = 1) = \sigma(\mathbf{v}^s \cdot (\mathbf{W}^s_h\mathbf{h}_i+\mathbf{W}^s_e\mathbf{E}[o_{i-1}] + \mathbf{W}^s_c\mathbf{c}_i + \mathbf{b}^s)) $$

ここで, $P(s_i=1)$はデコーダにおける$i^{th}$番目の時間ステップでスイッチがオンになる確率, $\mathbf{h}_i$は隠れ状態, $\mathbf{E}[o_{i-1}]$は前の時間ステップで発生した埋め込みベクトル, $\mathbf{c}_i$はアテンションで重み付けられた文脈ベクトル, $\mathbf{W},\mathbf{b},\mathbf{v}$はスイッチのパラメータである  
文書内に存在する単語の位置にアテンションの分布を, ポインタをサンプリングする分布として使用する  
  
$$ P^a_i(j) \propto exp(\mathbf{v}^a \cdot (\mathbf{W}^a_h \mathbf{h}_{i-1}+\mathbf{W}^a_e\mathbf{E}[o_{i-1}] + \mathbf{W}^a_c\mathbf{h}^d_j+\mathbf{b}^a) $$
$$ p_i = \arg max_j(P^a_i(j)) for i \in \{1, ... N_d\} $$
  
上記の数式において, $p_i$は要約におけるi番目の単語位置を示すポインタ値であり, これは文書の単語位置$j\in{1, ... , N_d}$上のアテンション分布$\mathbf{P^a_i}$からサンプリングされたものである  
また, $P^a_i(j)$はデコーダのi番目の時間ステップにおいて文書のj番目を指し示す確率であり, また$\mathbf{h}^d_j$は位置jにおけるエンコーダの隠れ状態を示す.  

学習時には, ターゲットの語彙に要約単語が存在しない場合に明示的にポインタ情報をモデルに与える  
要約で語彙外単語が文書の複数位置で発生した場合, その最初の発生箇所を優先し均衡を破る(？？？？？？？？？？？？)  
学習時には, 以下に示す条件付き対数尤度を最適化し, 正規化ペナルティを適用する  

$$log P(y|x) = \sum_i(g_ilog\{ P(y_i|\mathbf{y}-i,x)P(s_i) \}$$ $$ + (1-g_i)log\{ P(p(i)|\mathbf{y}-i,\mathbf{x})(1-P(s_i))\}$$

ここで, yとxはそれぞれ要約語と文書単語を示し, $g_i$は要約における位置iの語がデコーダ語彙に存在しない場合に常に0に設定される判別関数である  

テスト時, モデルは推定されたスイッチ確率$P(s_i)$にもとづいて, 単語を生成するかポインターを用いるかを各時間ステップで自動的に決定する  
提案モデルでは単に生成かポインターかの事後確率の最大を取り, 各時間ステップで最良の出力を生成する  
ポインタの機構は, 低頻度語のエンコーダにおける隠れ状態表現を用い, 文書からどの語を指すかを決定するため, 低頻度語の取り扱いにおいてより堅牢でありえる
隠れ状態は単語の文脈全体に依存するため, モデルは非出現単語を指し示すことはできても, それがターゲットの語彙に含まれることはない(あくまで現れるのはポインタであるため, 語彙に単語自体が存在するわけではない)  

### 2.4 階層的アテンションによる文書の階層構造の捕捉
ソース文書が非常に長いデータセットでは, 文書中のキーワードを特定するだけでなく, 要約を引き出せるような主要な文を特定することも重要となる  
この提案モデルでは, ソース側で2つの双方向RNNを使用して, 単語レベルと文章レベルで2つの重要度の概念を取り込むことに主眼を置いている  
アテンション機構は, 両方のレベルで同時に動作する  
単語レベルでのアテンションは, 対応する文レベルのアテンションにより再び重み付けされ, 以下に示すように再正規化される  

$$ P^a(j) = \frac{P^a_w(j)P^a_s(s(j))}{\sum^{N_d}_{k=1}P^a_w(k)(s(k))} $$

ここで, $P^a_w(j)$はソース文書におけるj番目の単語レベルでのアテンションの重み付けであり, s(j)はj番目の単語位置における文のODであり, $P^a_s(l)$はソース文書中の単語数であり, $P_a(j)$はj番目の単語位置で再スケール化されたアテンションである  
この最スケール化されたアテンションはデコーダの隠れ状態へ入力され, アテンション重み付き文脈ベクトルを計算するために用いられる  
さらに, 文書レベルRNNの隠れ状態に追加の位置埋め込みを連結し, 文書中の文の位置重要性をモデル化する  
このモデルを図3に示す  


## 3. 関連研究
またこんどね


## 4. 実験と結果
### 4.1 ギガワードコーパス
この一連の実験ではRushらの論文に記されているような, 注釈付きGigawordコーパスを使用した  
この研究の筆者が作成したスクリプトを使用しデータを前処理した結果, 約3.8Mの学習例が得られた  
このスクリプトでは約400Kのバリデーション(検定)とテストのサンプルも生成されるが, 本研究ではそのうち2000個のサンプルを検証用とテスト用にそれぞれランダムに抽出し作成した  

さらに, 提案モデルを精密に比較するため, Rushらが使用した精密なテストサンプルを取得した  
またスクリプトを少し修正し, トークン化された単語だけでなくシステム生成の品詞タグと名前付きエンティティタグも抽出した  

学習: 以下で説明するすべてのモデルについて, 同じコーパス上でモデル埋め込みを初期化するため200次元のword2vecベクトルを使用してモデルの埋め込みを初期化したが, 学習中にこれらの値を更新することを許可した  
エンコーダとデコーダの隠れ状態の次元は, すべての実験に置いて400に固定された  
Rushらが行ったように文書の最初の文のみをソースとして使用した場合, エンコーダの語彙サイズは119505であり, デコーダの(語彙)サイズは68885であった.  
学習にはAdadeltaを使用し, 初期学習レートを0.001に設定した  
バッチサイズは50を使用し, 各エポックでトレーニングデータをランダムにシャッフルしながら, トレーニングのスピードを上げるため10バッチごとに長さをソートした  
ドロップアウトや正規化は使用しなかったが, 勾配のクリッピングを適用した  
検証セットに基づき早期停止を用い, すべてのテスト性能番号を報告するため, 検証セット中で最良のモデルを使用した  
実験に用いるすべてのモデルには, 1エポックあたりの学習時間を約3倍に短縮するため, デコーダの語彙サイズを2000に制限するLarge-Vocaburally Trickを適用し, これとすべての後述モデルがフルボキャブラリに基づくモデルが要するエポックの50〜75%で収束するように作用する  

- **デコーディング:**  
デコード時には, サイズ5のビームサーチを用いて要約を生成し, 要約のサイズを最大30単語に限定した  
この最大サイズはサンプリングされた検証セット中に発見されたサイズである  
すべてのモデルの要約長(7.8〜8.3)の平均は, 特定のチューニングを行なわずとも, 検証セットの基準(約8.7)とほぼほぼ近しい値になっている  

- **計算コスト:**  
すべてのモデルをTesla40Kで学習したところ, 殆どのモデルは1エポックに12時間かかった階層的アテンションモデルを除いて平均で約10時間/1エポックを要した.
すべてのモデルはバリデーションコストに基づく早期終了基準を用いることで通常15エポック以内に収束した  
したがって, 収束までの実経過時間はモデルに応じて6-8日の間で異なってくる  
テスト時に要約を生成することは非常に高速であり, 1つのCPUでバッチサイズ1を使用して, 毎秒約20の要約の出力が得られる  

- **評価指標:**  
NallapaiらやChopraらと同様に, システムを評価するためRougeのフルレングスF1ヴァリアントを用いる  
制限長でのリコールはほとんどの先行研究で良い基準であるとされていたが, 短所の一つとして制限する長さをコーパスによって選択しなければならず, 研究者が性能を評価しづらい  
一方, フルレングスリコールは長さ制限を課すものではないが, 長めの要約について不公平に有利に働く  
フルレングスF1はこれらの問題を解決し, 特定の長さ制限を課すことなく, 長い要約にはペナルティを課すことができる  

加えて, ソースで発生するシステム要約文におけるトークンのパーセンテージも同様に報告する(表1におけるコピー率)  
すべての実験とギガワードコーパスでの結果を, 以下で説明する  

1. words-lvt2k-1sent:  
Large-Vocabulary Trickを持つベースラインのエンコーダー/デコーダーモデル  
このモデルは, Rushらが行ったようにソース文書の先頭文でのみ学習されている  

1. words-lvt2k-2sent:  
このモデルはソースの先頭2文を用いて学習しているという点を除いて, 前述のモデルと同一である  
このコーパスではソースに追加の文を加えることで性能が向上すると思われる  
そこで, 更に文を追加しようとしてみたところ, 性能は低下した。おそらくこのコーパスの後ろの文は要約に関与していないと思われる  

1. words-lvt2k-2sent-hieratt:  
ソース文書から2つの文を使用し, 2.4節で提案した階層的アテンションモデルを学習した  
表1に示すように, このモデルは先頭2つの文の相対的重要度を自動的に学習することにより, 階層的でないモデルと比較して性能が向上する  

1. feats-lvt2k-2sent:  
これまでと同様に先頭2文について学習するが, 注釈付きギガワードコーパスの品詞タグと名前付きエンティティタグ, さらにTF-IDF値を使用し, 2.2節で述べたようにソース側の入力埋め込みを補強する  
埋め込みベクトルは合計で元の100から155まで増加し, 表1に示すようにword-lvt2k-2sentと比較してよりよい性能をもたらし, このタスクにおける構文ベース特徴の有用性を確かめられた  

1. feats-lvt2k-2sent-ptr:  
2.3節で説明したスイッチング・ジェネレータ/ポインタモデルであるが, 上記のモデルのように文書側にもFeature-richな埋め込みを使用している  
実験では新しいモデルが表1に示す3つのRougeヴァリアントによるすべてのテストセットにおいて最高の性能を達成できることを示した  


- **State-of-the-artとの比較**  
Rushらが作成したテストサンプルについて, 提案モデルであるwords-lvt2k-1sentとstate-of-the-artなモデルを比較した  
また, LVTの語彙サイズが5kであるwords-lvt5k-1sentモデルを学習したが, ソースとターゲットの語彙サイズが400kと200kと非常に大きくなっている  

ここで, 最良のバリデーションモデルを評価しなかったのは, このテストセットがソース文書からの1文のみで構成されており, 最良モデルが必要とするNLPアノテーションが含まれていなかったからである  
それにも関わらず, 表は提案モデルがRushらのABS+モデルよりも優れていることを示している  
加えて, 提案モデルは表最後の列のsrc.コピー率に示されるように, より優れた集約能力を示す???????????????  
さらに, より大きなモデルであるwords-lvt5k-1sentは, Chopraらのstate-of-the-artなモデルよりもすぐれており, Rouge-1において統計的に有意な改善を示している  

ソースをモデル化するために用いた双方向RNNは, RushらやChopraらが畳込みアテンションエンコーダーで使用したbag-of-embeddingな表現よりも, あらゆる単語のより豊富な文脈情報を捉えていると考えている  
さらに, 複数のソース文, 単語レベルの言語的特徴, 必要に応じてソース単語へのポインタするスイッチ機構, また階層的アテンションといった重要な情報を明示的にモデリングすることは, それぞれが性能を飛躍的に向上させ, 要約における特定の問題を解決する  

### 4.2 DUCコーパス
DUCコーパスは2つのパートに分かれている。2003年のコーパスには624の文書と要約のペアが含まれ, 2004年のコーパスには500の対が含まれている  
これらのコーパスは大規模なニューラルネットワークを学習するには小さすぎるため, RushらはGigawordコーパスでモデルを学習したが, それに加えて2003年のDUCコーパスで学習した手作業の特徴選択による対数線形抽出要約モデルと組み合わせた  
Rushらは元のニューラルアテンションモデルをABSモデルと呼び, 結合後のモデルをABS＋とした  
また, ChopraらはこのコーパスにおけるRAS-Elmanモデルの性能を報告し, これは75バイトでのリコールのDUC指標により計測した結果非ニューラルネットワークベースの抽出・要約システムを含むこれまでのすべてのベースラインを上回っているため, 現在のstate-of-the-artである  

我々の実験では, DUC検証セットでチューニングを行うことなく, ギガワードコーパスで学習したモデルをそのまま実行する  
デコーダについて行った唯一の変更は, このコーパスにおける正式な評価が制限長のRougeリコールに基づくため, モデルが要約終了タグを出さないようにして, 各要約ごとに30文を強制的に出力させるようにしたことである  
このコーパスでも, ソースから供給されるのは単一文のみでありNLPアノテーションは含まれないため, words-lvt2k-1sentとwords-lvt5k-1sentのモデルのみを実行した  

テストセットにおけるこのモデルの性能は, ABSおよびABS+モデル, ChopraらのRAS-Elmanモデル, およびDUC2004における最高性能システムであるTOP-IARYと比較される  
提案手法の最良モデルであるwords-lvt5k-1sentモデルが, Rouge-1において拮抗した性能でありつつ, 3つのRougeヴァリアントのうち2つでRAS-Elmanよりも優れた性能であることは特筆すべき点である  

### 4.3 CNN/Daily Mainコーパス
ギガワードやDUCを含む既存の抽象文書要約コーパスは, 各要約につき一つの文のみで構成されている  
このセクションでは, 複数の文の要約を含む新しいコーパスの提示を行う  
このコーパスを生成するため, Hermannらによる節ベースの質問応答タスクに使用されている既存のコーパスを修正する  
この試みでは, 人手によって生成されたCNNとDaily Mailのウェブサイトにおける新しい話題の集約的な要約を, 質問として(エンティティの一つを秘匿して)使用し, もうええわ  



## 5. 定性分析
表5では, 最良モデルの一つであるfeats-lvt2k-2sentの検証セットから高品質な出力と低品質な出力を示している  
モデルがターゲット要約と異なる場合であっても, 要約は非常に意味があり関連性が高い傾向がある. これは, Rougeなどの単語/フレーズ一致の評価指標には反映されない事象である  
その一方, モデルは文章の意味を誤って解釈し, 表の劣悪な例に示すようなクッソおもろい解釈の要約を生成する  
このように, 複雑な文の｢意味｣を補足することはこれらのモデルにおける弱点といえる  
図4に示す出力例は, ギガワードコーパス上のスイッチングジェネレータ/ポインターモデルからの出力サンプルを示している  
これらの例からわかるように, モデルは名前付きエンティティだけでなく複数語句に対してもポインタを非常に正確に使用することがわかる  
ただし, その正確さにもかかわらず, モデル全体の性能改善はそこまでない  
このモデルの影響は低頻度語の分布が裾の重い分布である設定の場合により顕著になると考えられる  
